[< __INTRO MODULE 5__](./0.Introduction.ipynb)

---


# Index
- [Introduction](#introduction)
- [Parsing](#parsing)
    - [Different formats](#different-formats)
- [Creating a configuration file](#creating-a-configuration-file)
- [Interpolating](#interpolating)

---

# Introduction

Let's suppose that our application must connect to a database. This database has different environments which can be: `consolidation`, `pre-production` and `production`. Each environment consists of a replica of the same system but for a different purpose, for example, the consolidation and pre-production environments are intended for the development of new features and production to the final environment where the client connects.

Each of these environments will have its own credentials. So where should these credentials be stored?

This is where the `configparser` module comes in. This module will allow us to store the credentials for each environment securely.

An example format would be:

```INI
[DEFAULT]
# This is a comment.
host = localhost

[mariadb]
; Also this is a comment.
# LOOK THAT THE DEFINITION OF KEY/VALUE CAN BE DONE WITH : !!
name : hello
user : user
password : password

[redis]
port = 6379
db = 0
```

> __NOTE__: The configuration file has a format very similar to the Microsoft Windows INI.

In the example case we can identify the `DEFAULT`, `mariadb` and `redis` credentials. These groupings are known as sections (with the exception of DEFAULT).

---

# Parsing

As an example, let's say we want to get the credentials seen in the previous point (which we have saved in a file called [config.ini](./persistance/config.ini)) from a python script.

The steps to follow to read the information are:
1. Import `configparser`.
2. Instantiate `ConfigParser()`.
3. On the instantiated object, use the `read()` method passing as argument the path of the [file](./persistance/config.ini).
4. After performing the read, the object in point two will give us access to the information in the file.

The following is an example of reading the file:

In [18]:
import configparser

obj = configparser.ConfigParser()
print(obj.read('./persistance/config.ini'))

print(obj.sections())

print('mariadb section:')
print('Host:', obj['mariadb']['host'])
print('Database:', obj['mariadb']['name'])
print('Username:', obj['mariadb']['user'])
print('Password:', obj['mariadb']['password'], '\n')

print('redis section:')
print('Host:', obj['redis']['host'])
print('Port:', int(obj['redis']['port']))
print('Database number:', int(obj['redis']['db']))

['./persistance/config.ini']
['mariadb', 'redis']
mariadb section:
Host: localhost
Database: hello
Username: user
Password: password 

redis section:
Host: dummy
Port: 6379
Database number: 0


As we can see, the `host` field in the `DEFAULT` section is accessible from both `mariadb` and `mariadb`. This is because the `DEFAULT` fields are replicated in other environments (unless they are explicitly modified as we see in redis).


---

## Different formats

Apart from the format (similar to Microsoft Windows INI), `configparser` also allows the use of other formats.

The formats that could be read are:
- `read_string`: Is able to interpret a string.
- read_file`: Is able to read a file.
    - It differs from read in that this method reads an object that is already a file (instead of fetching it directly through the `path` as `read()` does).
- read_dict`: Is capable of interpreting a dictionary.

As an example, it will read a file from a dictionary. The only difference is that instead of using the `read()` method to read the file we will use the `read_dict()` method.

Here is an example:

In [12]:
obj2 = configparser.ConfigParser()
obj2.read_dict(
    {
        'DEFAULT': {
            'host': 'localhost'
            },
        'mariadb': {
            'name': 'hello 2',
            'user': 'root',
            'password': 'password'
            },
        'redis': {
            'port': 6379,
            'db': 0
            }
        }
    )

for section in obj2.sections():
    print(section.center(50, "*"))
    for key, value in obj2[section].items():
        print(f'{key}: {value}')

*********************mariadb**********************
name: hello 2
user: root
password: password
host: localhost
**********************redis***********************
port: 6379
db: 0
host: localhost



---

# Creating a configuration file

To create a configuration file, you should treat the `ConfigParser` object as a `dictionary`. Note that the section names are keys, while their options are listed in separate dictionaries.

The above configuration is saved using the write method, which requires an open file to be passed in text mode. For this purpose, the built-in open method is used.

> __NOTE__: You can edit config files also when reading the file

As following we will show how to create these configuration:

In [14]:
import configparser

config = configparser.ConfigParser()

config['DEFAULT'] = {'host': 'localhost'}
config['mariadb'] = {
    'name': 'byee',
    'user': 'user1',
    'password': '123456'
    }
config['redis'] = {
    'port': 6379,
    'db': 0}

path_config = './persistance/created_config.ini'

with open(path_config, '+w') as configfile:
    config.write(configfile)


obj = configparser.ConfigParser()
obj.read(path_config)

for section in obj.sections():
    print(section.center(50, "*"))
    for key, val in obj[section].items():
        print(f"{key}: {val}")

*********************mariadb**********************
name: byee
user: user1
password: 123456
host: localhost
**********************redis***********************
port: 6379
db: 0
host: localhost



---

# Interpolating

The big advantage of the configuration file is the possibility of using interpolation. It allows you to create expressions consisting of a placeholder under which the appropriate value will be substituted.

Take a look at the configuration file below:

```INI
[DEFAULT]
host = localhost

[mariadb]
name = hello
user = user
password = password

[redis]
port = 6379
db = 0
dsn = redis://%(host)s
```

The configuration file has been extended with another option called dsn. Its value contains the placeholder `%(host)s`, which needs to be replaced by an appropriate value.

> __NOTE__: Placing any key between `%` and `s` informs the parser of the need to interpolate. Of course, all the work is done for us, and we only get the ready results.

Let's check if these works:

In [16]:
import configparser

config = configparser.ConfigParser()

config['DEFAULT'] = {'host': 'localhost'}
config['mariadb'] = {
    'name': 'byee',
    'user': 'user1',
    'password': '123456'
    }
config['redis'] = {
    'port': 6379,
    'db': 0,
    'dns': 'redis://%(host)s',
    'direct_link': '192.168.1.1:%(port)s'}

path_config = './persistance/created_config.ini'

with open(path_config, '+w') as configfile:
    config.write(configfile)


obj = configparser.ConfigParser()
obj.read(path_config)

for section in obj.sections():
    print(section.center(50, "*"))
    for key, val in obj[section].items():
        print(f"{key}: {val}")

*********************mariadb**********************
name: byee
user: user1
password: 123456
host: localhost
**********************redis***********************
port: 6379
db: 0
dns: redis://localhost
direct_link: 192.168.1.1:6379
host: localhost



---

[< __INTRO MODULE 5__](./Introduction.ipynb)